# MCP Historical Weather Comparison

<a target="_blank" href="https://colab.research.google.com/github/tylere/claude-mcp-historical-weather/blob/main/demo.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Overview

**Imagine asking an AI assistant**: *"Which city had more pleasant weather last year - Portland or Phoenix? Show me temperature patterns, rainfall data, and create visualizations comparing their climates."*

Most AI systems today would give you general knowledge about these cities' typical weather patterns, perhaps mentioning that Phoenix is hot and dry while Portland is mild and rainy. But they can't access specific historical data, calculate exact statistics, or create custom visualizations from real weather records.

**This is the fundamental limitation we're solving today.**

### The Problem: AI's Knowledge Boundaries

Large Language Models (LLMs) like Claude are trained on vast amounts of text data - typically 10-100 terabytes after processing. They excel at understanding language, reasoning about concepts, and providing general knowledge. However, they face two critical limitations:

1. **Knowledge Cutoff**: They can't access information beyond their training date or real-time data
2. **Specific Data Analysis**: They struggle with custom analysis of specialized datasets that weren't in their training

When you ask Claude about weather patterns between specific cities for particular years, it can only provide general climatic knowledge - not the precise, data-driven analysis you actually need.

### The Solution: Expanding AI's Reach

**Model Context Protocol (MCP)** bridges this gap by giving AI systems secure access to external data sources and tools. Think of it as providing AI with a telescope to see beyond its training horizon, or giving it specialized instruments to perform precise measurements.

Instead of being limited to static knowledge, MCP-enabled AI can:
- **Access live data** from APIs and databases
- **Perform custom calculations** on specific datasets  
- **Generate visualizations** from real-world information
- **Provide current, precise answers** to data-driven questions

### The Scale of Opportunity

Consider the vast landscape of specialized data that exists beyond typical AI training:

Modern LLMs are trained on 10-100 TB of carefully curated text data. But domain-specific datasets dwarf this scale, containing rich information that could revolutionize how we interact with data:

| Dataset | Description | Size (terabytes) |
| ---  | --- | --- |
| [Open-Meteo Weather](https://open-meteo.com/en/docs/historical-weather-api) | Historical weather data back to 1940 | 90 |
| [Landsat Collection 2](https://www.usgs.gov/landsat-missions/landsat-commercial-cloud-data-access) | 5+ decades of satellite imagery | 9,000 |
| [Sequence Read Archive (SRA)](https://www.ncbi.nlm.nih.gov/sra/docs/) | Global DNA and RNA sequencing data | 36,000 |

These massive, specialized datasets are typically locked behind complex APIs and technical barriers. MCP democratizes access by allowing AI to serve as an intelligent interface to this information.

### What You'll Build Today

This notebook demonstrates MCP integration using historical weather data as our example domain. You'll create a system where Claude can:

- **Query real weather data** spanning decades for any global location
- **Calculate custom climate statistics** (temperature ranges, precipitation patterns, sunshine duration)  
- **Generate beautiful visualizations** comparing weather patterns between cities
- **Provide data-driven insights** about climate trends and patterns

By the end, you'll have transformed Claude from a knowledge-limited chatbot into a powerful weather analysis assistant with access to 80+ years of global climate data.

### What You'll Learn

This tutorial will teach you to:

1. **Understand MCP's Value**: See exactly what AI can accomplish when connected to real-world data
2. **Implement MCP Integration**: Build the technical bridge between Claude and external APIs
3. **Design Data Analysis Tools**: Create functions that transform raw data into insights
4. **Enable Interactive Analysis**: Let users ask natural language questions about complex datasets

### Why This Matters

Weather data is universally relatable - everyone has opinions about climate and can immediately understand the value of precise weather comparisons. But the same MCP principles apply to any domain:

- **Scientific Research**: Connect AI to genomic databases, astronomical catalogs, or experimental datasets
- **Business Intelligence**: Link AI to sales data, market research, or customer analytics  
- **Environmental Monitoring**: Access sensor networks, satellite imagery, or ecological databases
- **Healthcare**: Query medical databases, research papers, or treatment outcomes

This weather demo serves as your blueprint for building AI systems that can intelligently interface with any specialized dataset.

**Ready to give AI superpowers? Let's begin.** 🚀

## Setup

This section will get your environment ready for MCP integration. We'll handle both Google Colab and local Jupyter setups automatically.

### Colab Setup (if needed)

The following cell automatically detects your environment and installs the necessary dependencies for both Google Colab.

In [ ]:
import sys

# Check if we're running in Google Colab
IN_COLAB = 'google.colab' in sys.modules
print(f"Running in {'Google Colab' if IN_COLAB else 'local Jupyter environment'}")

# Install required packages
if IN_COLAB:
    # Define packages that match pyproject.toml dependencies
    packages = [
        "altair>=5.5.0",
        "anthropic>=0.66.0",
        "ipykernel>=6.30.1",
        "mcp>=1.13.1",
        "numpy>=2.2.6",
        "openmeteo-requests>=1.7.2",
        "pandas>=2.3.2",
        "requests>=2.32.5",
        "requests-cache>=1.2.1",
        "retry-requests>=2.0.0",
    ]
    
    print("Installing required packages...")
    for package in packages:
        print(f"Installing {package}...")
        !pip install {package}
    
    print("✓ All packages installed successfully!")

### Python Package Imports

Now let's import all the necessary packages for our weather comparison system.

In [ ]:
from datetime import datetime
import os
import json
from pprint import pprint

import getpass

# Data handling and validation
from pydantic import BaseModel

# Visualization
import altair as alt

# For displaying rich notebook content
from IPython.display import Markdown, display

# Claude API
import anthropic

import requests_cache  # for caching API responses
from retry_requests import retry  # for retrying API requests
import openmeteo_requests  # for making API requests to Open-Meteo

import pandas as pd  # for data manipulation

print("All packages imported successfully!")

### Anthropic API Key Setup

To interact with Claude, we need your Anthropic API key. The function below securely retrieves it using the safest available method:

1. **Environment variable** (`ANTHROPIC_API_KEY`)
2. **Colab secrets** (if in Google Colab)  
3. **Secure input prompt** (hidden, not stored in notebook)

In [ ]:
def get_api_key():
    try:
        if IN_COLAB:
            # Import package for accessing user data
            from google.colab import userdata
            api_key = userdata.get('ANTHROPIC_API_KEY')
        else:
            api_key = os.environ.get("ANTHROPIC_API_KEY")
    except:
        # Prompt user for their API key
        api_key = getpass.getpass("Enter your Anthropic API key: ")
    return api_key

### Verbose Output Control

Set this flag to control debugging information during MCP tool execution. Keep it `False` for cleaner output.

In [ ]:
verbose_output = False

## Motivation: Why Do We Need to Extend LLMs?

Let's start with an experiment. We'll ask Claude to analyze weather data **without giving it access to any external tools or data sources**.

This demonstration will reveal the fundamental limitation that MCP solves: LLMs can only work with information from their training data, which has knowledge cutoffs and lacks access to specific, current datasets.

In [ ]:
weather_question = """
Please compare the annual weather statistics for the years 2000-2023
for San Francisco, Seattle, and Austin, and Redwood City. 

I'd like to see:
1. Days where the maximum temperature is between 18 and 24 degrees Celsius
2. Days where the precipitation exceeds 10 mm
3. Days where the humidity is between 30 and 60%

Please provide specific data and create a comparison showing which city had more favorable weather conditions.
"""

<div class="alert alert-block alert-info">
<b>Note:</b> Why Redwood City?

The city's slogan is *"Climate Best By Government Test"*. According to [wikipedia](https://en.wikipedia.org/wiki/Redwood_City,_California#Slogan), this is based on a climatological survey conducted by the United States and German governments prior to World War I. So it seems like a good baseline city for comparisons in terms of climate.

<a title="Coolcaesar, CC BY 4.0 &lt;https://creativecommons.org/licenses/by/4.0&gt;, via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:Redwood_City_Western_Arch.jpg"><img alt="The arch at the western end of downtown Redwood City, California with the city slogan, Climate Best by Government Test." src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Redwood_City_Western_Arch.jpg/1024px-Redwood_City_Western_Arch.jpg?20250816212854"></a>
</div>

Let's ask Claude our weather question, without using any extra tools.

In [ ]:
def ask_claude_without_tools(question: str) -> str:
    """
    Send a question to Claude without any external tools or data access.
    
    Args:
        question: The question to ask Claude
    
    Returns:
        Claude's response as a string
    """
    try:
        client = anthropic.Anthropic(api_key=get_api_key())
        message = client.messages.create(
            model="claude-sonnet-4-20250514",
            max_tokens=1000,
            messages=[
                {
                    "role": "user", 
                    "content": question
                }
            ]
        )
        return message.content[0].text
    except Exception as e:
        return f"Error: {e}"

# Test Claude's response without external data
print("🤖 Asking Claude about weather data WITHOUT external tools...")
print("="*70)
print(f"Question: {weather_question}")
print("="*70)
print("Claude's Response:")
print()

response = ask_claude_without_tools(weather_question)
display(Markdown(response))

<div class="alert alert-block alert-info">
<b>Note:</b> The LLM model (without tools) returns some general context, but we can do better than that!
</div>

As you can see from Claude's response above, without access to external data sources, the model has several limitations:

1. **No real-time data**: Claude can't access current or specific historical weather data
2. **General knowledge only**: Responses are based on general patterns from training data
3. **No specific metrics**: Can't provide exact precipitation amounts, temperatures, or day counts
4. **No visualizations**: Can't create charts or graphs from actual data

These gaps can be addressed by custom tools and the **Model Context Protocol (MCP)** - they can be used to bridge the gap between the model's reasoning capabilities and real-world data access.

## Analysis & Data Processing

In this section, we'll create the core Python functions that power our weather analysis system:

- **Retrieve** daily weather data from the Open-Meteo API for multiple locations
- **Aggregate** daily data into meaningful annual statistics
- **Visualize** trends with interactive time-series charts

### Location Data Model

We start by defining a simple `Location` class using Pydantic for data validation. This ensures our coordinates are properly formatted for the weather API.

In [ ]:
class Location(BaseModel):
    name: str
    longitude: float
    latitude: float

## Weather Data Retrieval

Next we'll configure access to the [Open-Meteo Historical Weather API](https://open-meteo.com/), including intelligent caching and retry mechanisms to avoid unnecessary API calls.

In [ ]:
api_url = "https://archive-api.open-meteo.com/v1/archive"

weather_variables = [
    "temperature_2m_max",
    "temperature_2m_mean",
    "temperature_2m_min",
    "relative_humidity_2m_max",
    "relative_humidity_2m_mean",
    "relative_humidity_2m_min",
    "rain_sum",
    "precipitation_hours",
    "sunshine_duration",
  ]

# Setup an Open-Meteo API client with a cache and retry mechanism
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

<div class="alert alert-block alert-info">
<b>Note:</b> The Open-Meteo API offers free access for non-commercial use. If you want to use the API commercially (or just want to support the project) consider <a href="https://open-meteo.com/en/pricing">purchasing an commercial use license</a>. 
</div>

Next we'll define a function to retrieve weather data from one or more locations, returning the data as a pandas DataFrame.

In [ ]:
def get_weather_data(
      locations: list[Location],
      start_date: str,
      end_date: str,
      variables: list[str]
    ):
    """
    Get weather data for one or more locations.
    
    Args:
        locations: List of Location objects
        start_date: Start date in YYYY-MM-DD format
        end_date: End date in YYYY-MM-DD format
        variables: List of weather variables to retrieve
    
    Returns:
        Pandas DataFrame with weather data
    """
    
    def parse_response(variables, response, location_name):
        daily = response.Daily()
        daily_data_dict = {
            "date": pd.date_range(
            start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
            end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
            freq = pd.Timedelta(seconds = daily.Interval()),
            inclusive = "left"
            )
        }

        # Add the variable data.
        for i, variable in enumerate(variables):
            daily_data_dict[variable] = daily.Variables(i).ValuesAsNumpy()

        # Add a column for the location name.
        daily_data_dict["location_name"] = location_name

        return pd.DataFrame(daily_data_dict)

  
    params = {
        "latitude": [x.latitude for x in locations],
        "longitude": [x.longitude for x in locations],
        "start_date": start_date,
        "end_date": end_date,
        "daily": variables,
    }

    # Query for weather data and get one response per location.
    responses = openmeteo.weather_api(api_url, params=params)

    # Concatenate all of the responses into a single dataframe
    daily_df_list = [parse_response(variables, response, locations[i].name) for i, response in enumerate(responses)]
    daily_df = pd.concat(daily_df_list, axis=0)

    return daily_df

### 🧪 **Try It Out**

Let's test our function with real data from two Bay Area cities to see the weather data retrieval in action.

In [ ]:
test_locations = [
  Location(
    name='San Francisco',
    latitude=37.7749,
    longitude=-122.4194,
  ),
  Location(
    name='Redwood City',
    latitude=37.4848,
    longitude=-122.2281,
  )
]

# Test the original functionality
daily_data = get_weather_data(
  locations=test_locations,
  start_date="2000-01-01",
  end_date="2019-12-31",
  variables=['temperature_2m_mean', 'temperature_2m_max', 'rain_sum', 'sunshine_duration']
)
daily_data

<div class="alert alert-block alert-info">
<b>Note:</b> Sometimes the free Open-Meteo API gets overloaded and times out.
If this happens, take it as a suggestion to stand up and stretch for a few minutes. 
</div>

## Calculate Annual Statistics

To enable better long-term comparisons, we can write a function that calculates annual statistics of how many times a variable exceeds a minimum or maximum threshold.

Examples:
- Days that the maximum temperature exceeds 30 degrees C
- Days that the mean temperatures is between 20 and 25 degrees C
- Days that rain exceeds 2 mm 

In [ ]:
def calculate_annual_stats(
    daily_data: pd.DataFrame,
    variable: str,
    threshold_min: float = None,
    threshold_max: float = None
    ) -> pd.DataFrame:
  """
  Calculate annual statistics for the given daily data.
  
  Args:
    daily_data: DataFrame containing weather data
    variable: Name of the variable column to analyze
    threshold_min: Optional minimum threshold (inclusive)
    threshold_max: Optional maximum threshold (inclusive)
  
  Returns:
    DataFrame with columns: year, count, location_name
  """
  # Validate threshold parameters
  if threshold_min is None and threshold_max is None:
    raise ValueError("At least one of threshold_min or threshold_max must be provided")
  
  # Get unique location names from the daily data
  locations = daily_data['location_name'].unique()
  
  # Initialize list to store results
  results = []
  
  # Process each location
  for location in locations:
    # Filter data for this location and make an explicit copy to avoid SettingWithCopyWarning
    location_data = daily_data[daily_data['location_name'] == location].copy()
    
    # Extract year from date column
    location_data['year'] = pd.to_datetime(location_data['date']).dt.year
    
    # Apply threshold filters
    def count_days_in_range(x):
      mask = pd.Series(True, index=x.index)  # Start with all True
      
      if threshold_min is not None:
        mask = mask & (x >= threshold_min)
      
      if threshold_max is not None:
        mask = mask & (x <= threshold_max)
      
      return mask.sum()
    
    yearly_counts = location_data.groupby('year')[variable].apply(count_days_in_range)
    
    # Convert to dataframe
    yearly_df = pd.DataFrame({
      'year': yearly_counts.index,
      'count': yearly_counts.values,
      'location_name': location
    })
    
    results.append(yearly_df)
    
  # Combine all results
  return pd.concat(results, axis=0).reset_index(drop=True)

### 🧪 **Try It Out**

Let's test our annual statistics function with the weather data we just retrieved.

In [ ]:
annual_stats = calculate_annual_stats(
    daily_data,
    variable='temperature_2m_max',
    threshold_min=20,
    threshold_max=25
)
annual_stats.head()

## Time-Series Visualization

Interpreting a long table of numbers can be pretty challenging for humans, so let's create a function for charting the data.

In [ ]:
def create_annual_stats_chart(
    annual_stats: pd.DataFrame,
    title: str
  ) -> alt.Chart:
  """
  Create a chart showing the annual statistics.
  """
  # Base chart with data points
  base = alt.Chart(annual_stats).encode(
    x='year:O',
    y='count:Q',
    color='location_name:N'
  )

  # Create line chart
  lines = base.mark_line()

  # Add trend lines
  trend_lines = base.transform_regression(
    'year', 'count', 
    groupby=['location_name']
  ).mark_line(
    strokeDash=[5,5]
  ).encode(
    color='location_name:N'
  )

  # Combine the line and trend lines
  return (lines + trend_lines).properties(
    title=title
  )

### 🧪 **Try It Out**

Let's create an interactive chart from our annual statistics to visualize the trends.

In [ ]:
create_annual_stats_chart(
    annual_stats,
    title='Days with Temperature Mean between 20 and 25 degrees C'
)

Now that we have functions to retrieve, analyze, and format weather statistics. In the following section we will expose that functionality to Claude.

# Model Context Protocol (MCP) Integration

Now we'll connect our weather analysis functions to Claude using the [Model Context Protocol (MCP)](https://modelcontextprotocol.io/). 

We'll build an MCP client that can:
- **Register tools** with Claude
- **Execute functions** when Claude needs data
- **Process results** and format them for display
- **Manage conversations** with full tool integration

In [ ]:
class MCPClient:
    """A client for interacting with Claude using the Model Context Protocol (MCP).
    
    This class handles registering tools, executing them, and managing conversations with 
    Claude using the MCP format. It processes both text and image outputs from tools
    and formats them appropriately for Claude's consumption.

    Attributes:
        client: The Anthropic client instance
        tools: List of registered MCP tools
        tool_functions: Dictionary mapping tool names to their implementation functions
    """
    def __init__(self, api_key):
        self.client = anthropic.Anthropic(api_key=api_key)
        self.tools = []
        self.tool_functions = {}
    
    def register_tool(self, name, description, input_schema, function):
        """Register an MCP tool"""
        tool = {
            "name": name,
            "description": description,
            "input_schema": input_schema
        }
        self.tools.append(tool)
        self.tool_functions[name] = function
    
    def execute_tool(self, tool_name, tool_input):
        """Execute a registered tool"""
        if tool_name in self.tool_functions:
            return self.tool_functions[tool_name](**tool_input)
        else:
            return f"Tool {tool_name} not found"
    
    def _process_mcp_tool_result(self, tool_result_json: str):
        """Process MCP tool result and return content for Claude"""
        try:
            result = json.loads(tool_result_json)
            if "error" in result:
                return [{"type": "text", "text": f"Error: {result['error']}"}]
            
            content = []
            
            # Add text content
            if "text" in result:
                content.append({
                    "type": "text",
                    "text": result["text"]
                })
            
            if "location" in result:
                content.append({
                    "type": "location",
                    "text": f"Location: {result['location']}"
                })

            # Add image content 
            if "chart_json" in result:
                content.append({
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": "application/vnd.vegalite.v5+json",
                        "data": result["chart_json"]
                    }
                })
            return content
        
        except json.JSONDecodeError:
            # If not JSON, treat as plain text
            return [{"type": "text", "text": tool_result_json}]

    def chat_with_tools(self,
                        user_message,
                        model="claude-sonnet-4-20250514",
                        max_iterations=10):
        """Send messages and handle tool calls automatically"""
        
        # Initialize the messages list
        messages = [{"role": "user", "content": user_message}]

        # Track images generated by tools throughout the conversation
        tool_images = []

        for iteration in range(max_iterations):
            if verbose_output:
                display(Markdown(f"`chat_with_tools()`: Iteration {iteration + 1} --------------------------"))
            try:
                if verbose_output:
                    display(Markdown(f"`chat_with_tools()`: messages ="))
                    pprint(messages)
                # Make request to Claude with tools
                response = self.client.messages.create(
                    model=model,
                    max_tokens=4096,
                    tools=self.tools,
                    messages=messages
                )
                if verbose_output:
                    display(Markdown(f"`chat_with_tools()`: response.content ="))
                    for block in response.content:
                        pprint(block)
                
                # Add assistant's response to conversation
                messages.append({"role": "assistant", "content": response.content})

                if verbose_output:
                    display(Markdown(f'`chat_with_tools()`: {response.stop_reason = }'))
                if response.stop_reason == "tool_use":
                    # Process all tool uses via MCP
                    tool_results = []
                    
                    for block in response.content:
                        if block.type == "tool_use":

                            mcp_result = self.execute_tool(block.name, block.input)
                            content = self._process_mcp_tool_result(mcp_result)

                            # Extract and store any images from tool results
                            image_free_content = []
                            for content_item in content:
                                if content_item['type'] == 'image':
                                    tool_images.append(content_item)
                                else:
                                    image_free_content.append(content_item)

                            tool_results.append({
                                "type": "tool_result",
                                "tool_use_id": block.id,
                                "content": image_free_content
                            })
                    
                    # Add tool results to conversation
                    messages.append({"role": "user", "content": tool_results})
                    
                    # Continue conversation loop
                    continue
                    
                else:
                    # Final response - display both text and any tool-generated images
                    
                    # Create a custom response object that includes tool images
                    class ResponseWithImages:
                        def __init__(self, original_response, tool_images):
                            self.content = []
                            
                            # Add original text content
                            for block in original_response.content:
                                self.content.append(block)
                            
                            # Add tool-generated images as additional content
                            for image in tool_images:
                                # Convert tool image format to Claude response format
                                image_block = type('ImageBlock', (), {
                                    'type': 'image',
                                    'source': type('Source', (), {
                                        'data': image['source']['data'],
                                        'media_type': image['source']['media_type']
                                    })()
                                })()
                                self.content.append(image_block)
                    
                    # Return enhanced response with images
                    return ResponseWithImages(response, tool_images)
            
            except Exception as e:
                print(f"Error in iteration {iteration + 1}: {e}")
                return None
        
        print(f"\nReached maximum iterations ({max_iterations})")
        return None

Next we'll define an MCP tool for comparing annual weather statistics at one or more locations.

In [ ]:
def compare_locations_mcp(
    locations: list,
    variable: str,
    threshold_min: float = None,
    threshold_max: float = None,
    start_year: int = None,
    end_year: int = None
) -> str:
    """
    Compare the annual weather statistics for one or more locations.
    
    Args:
        locations: List of locations to analyze and compare
        variable: Weather variable to compare
        threshold_min: Optional minimum threshold (inclusive)
        threshold_max: Optional maximum threshold (inclusive)
        start_year: Start year (default: 50 years before end_year)
        end_year: End year (default: previous year)
        
    Returns:
        JSON string containing weather comparison text and chart data
    """
    try:
        # Validate input
        if not locations or len(locations) == 0:
            return json.dumps({"error": "At least one location must be provided"})
        
        # Convert dictionaries to Location objects
        location_objects = []
        for loc in locations:
            if isinstance(loc, dict):
                location_objects.append(Location(
                    name=loc["name"],
                    latitude=loc["latitude"],
                    longitude=loc["longitude"]
                ))
            else:
                location_objects.append(loc)

        # Set default year range if not provided
        if end_year is None:
            end_year = datetime.now().year - 1  # Default to previous year
        if start_year is None:
            start_year = end_year - 50  # Default to 50 years before end_year
        
        # Fetch weather data for all locations
        weather_data = get_weather_data(
            locations=location_objects,
            start_date=f"{start_year}-01-01",
            end_date=f"{end_year}-12-31",
            variables=[variable]
        )
                
        # Calculate annual statistics if thresholds were provided
        if threshold_min is not None or threshold_max is not None:
            annual_stats = calculate_annual_stats(
                daily_data=weather_data,
                variable=variable,
                threshold_min=threshold_min,
                threshold_max=threshold_max
            )
            
            # Create chart
            threshold_desc = ""
            if threshold_min is not None and threshold_max is not None:
                threshold_desc = f"Days of {variable} between {threshold_min} and {threshold_max}"
            elif threshold_min is not None:
                threshold_desc = f"Days of {variable} above {threshold_min}"
            elif threshold_max is not None:
                threshold_desc = f"Days of {variable} below {threshold_max}"
            
            chart = create_annual_stats_chart(annual_stats, threshold_desc)
        else:
            # Just show raw data comparison without thresholds
            annual_stats = None
            chart = None
        
        # Generate comparison text for all locations
        comparison_text = f"🌍 Location Analysis: {', '.join([loc.name for loc in location_objects])}\n\n"
        comparison_text += f"📊 {variable.replace('_', ' ').title()} Analysis:\n\n"
        
        # Statistics for each location
        location_stats = []
        for loc in location_objects:
            loc_data = weather_data[weather_data['location_name'] == loc.name]
            if not loc_data.empty:
                avg_val = loc_data[variable].mean()
                min_val = loc_data[variable].min()
                max_val = loc_data[variable].max()
                
                location_stats.append({
                    'name': loc.name,
                    'avg': avg_val,
                    'min': min_val,
                    'max': max_val,
                    'coordinates': f"{loc.latitude}, {loc.longitude}"
                })
                
                comparison_text += f"🏙️ **{loc.name}**\n"
                comparison_text += f"• Average: {avg_val:.1f}\n"
                comparison_text += f"• Range: {min_val:.1f} to {max_val:.1f}\n" 
                comparison_text += f"• Coordinates: {loc.latitude}, {loc.longitude}\n\n"

        # Add threshold-based analysis if applicable
        if annual_stats is not None:
            comparison_text += "🎯 **Threshold Analysis:**\n"
            
            # Calculate average days per location
            for loc in location_objects:
                loc_stats = annual_stats[annual_stats['location_name'] == loc.name]
                if not loc_stats.empty:
                    avg_days = loc_stats['count'].mean()
                    comparison_text += f"• {loc.name}: {avg_days:.0f} days per year on average\n"
            
            comparison_text += f"• Criteria: {threshold_desc}\n\n"
        
        # Add summary comparison if more than one location
        if len(location_objects) > 1:
            comparison_text += "🏆 **Summary:**\n"
            
            # Find best and worst for average values
            if location_stats:
                best_avg = max(location_stats, key=lambda x: x['avg'])
                worst_avg = min(location_stats, key=lambda x: x['avg'])
                
                comparison_text += f"• Highest average {variable.replace('_', ' ')}: {best_avg['name']} ({best_avg['avg']:.1f})\n"
                comparison_text += f"• Lowest average {variable.replace('_', ' ')}: {worst_avg['name']} ({worst_avg['avg']:.1f})\n"
                
                # Add threshold analysis summary if available
                if annual_stats is not None:
                    threshold_stats = []
                    for loc in location_objects:
                        loc_stats = annual_stats[annual_stats['location_name'] == loc.name]
                        if not loc_stats.empty:
                            avg_days = loc_stats['count'].mean()
                            threshold_stats.append({'name': loc.name, 'avg_days': avg_days})
                    
                    if threshold_stats:
                        best_threshold = max(threshold_stats, key=lambda x: x['avg_days'])
                        comparison_text += f"• Most days meeting criteria: {best_threshold['name']} ({best_threshold['avg_days']:.0f} days/year)\n"

        result = {"text": comparison_text}
        if chart:
            result["chart_json"] = chart.to_dict()
            
        return json.dumps(result)
        
    except Exception as e:
        return json.dumps({"error": f"Location analysis failed: {str(e)}"})

## 🧪 **Try It Out**

Start by creating an instance of the MCP client.

In [ ]:
mcp_client = MCPClient(get_api_key())

Next, register our tool with the MCP client.

In [ ]:
mcp_client.register_tool(
    name="compare_locations_mcp",
    description="Analyze and compare annual weather statistics for one or more cities with optional threshold analysis and visualization.",
    input_schema={
            "type": "object",
            "properties": {
                "locations": {
                    "type": "array",
                    "description": "List of locations to analyze and compare",
                    "items": {
                        "type": "object",
                        "properties": {
                            "name": {"type": "string"},
                            "latitude": {"type": "number"},
                            "longitude": {"type": "number"}
                        },
                        "required": ["name", "latitude", "longitude"]
                    },
                    "minItems": 1
                },
                "variable": {
                    "type": "string",
                    "description": "Weather variable to compare",
                    "enum": weather_variables,
                },
                "threshold_min": {
                    "type": "number",
                    "description": "Optional minimum threshold for counting days (inclusive)"
                },
                "threshold_max": {
                    "type": "number", 
                    "description": "Optional maximum threshold for counting days (inclusive)"
                },
                "start_year": {
                    "type": "integer",
                    "description": "Start year (default: 50 years before end_year)"
                },
                "end_year": {
                    "type": "integer",
                    "description": "End year (default: previous year)"
                }
            },
            "required": ["locations"]
        },
    function=compare_locations_mcp
)

We can verify that the MCP tool is configured properly, by executing the tool directly.

In [ ]:
tool_results_str = mcp_client.execute_tool(
  tool_name="compare_locations_mcp",
  tool_input={
    "locations": [
        {
            "name": "San Francisco",
            "latitude": 37.7749,
            "longitude": -122.4194
        },
        {
            "name": "Seattle",
            "latitude": 47.6062,
            "longitude": -122.3321
        }
    ],
    "variable": "temperature_2m_max",
    "threshold_min": 18,
    "threshold_max": 24,
    "start_year": 2000,
    "end_year": 2019
})

tool_results = json.loads(tool_results_str)

# Display the tool results
display(Markdown(tool_results['text']))
if 'chart_json' in tool_results:
  display(alt.Chart.from_dict(tool_results['chart_json']))

## Display Helper Function

We need one final utility function to properly display Claude's responses in our notebook. This function handles both text content and interactive visualizations.

In [ ]:
def display_claude_response(response):
    """Display Claude response with both text and images"""    
    if response:
        for content in response.content:
            if content.type == "text":
                display(Markdown(content.text))
            elif content.type == "image":
                if content.source.media_type == "application/vnd.vegalite.v5+json":
                    # Recreate the chart from JSON dictionary
                    altair_chart = alt.Chart.from_dict(content.source.data)
                    display(altair_chart)


# MCP in Action: Querying with Enhanced Claude

Now for the exciting part! Let's see how Claude performs with our MCP tools compared to its earlier limitations.

### Revisiting Our Weather Question

First, let's review the complex weather question we asked earlier (without tools):

In [ ]:
print(weather_question)

<div class="alert alert-block alert-success">
<b>Tip:</b> Update this question ask for locations that are relevant to you!<br/>
</div>

Now let's ask Claude the same question, this time using the MCP tool we created.

In [ ]:
response = mcp_client.chat_with_tools(weather_question)

In [ ]:
display_claude_response(response)

### 🎉 **The Transformation**

Notice the dramatic difference! With MCP integration, Claude now provides:
- **Specific numerical data** from real weather records
- **Custom analysis** tailored to your exact criteria  
- **Interactive visualizations** generated from actual datasets
- **Data-driven insights** impossible without external data access

# 🚀 Where to Next?

You now have a powerful foundation for connecting AI to external data sources. Here are some exciting directions to explore:

## Expand Weather Analysis
- Implement additional [ETCCDI Climate Change Indices](https://etccdi.pacificclimate.org/list_27_indices.shtml) compiled by [CLIVAR](https://www.clivar.org/)
- Add weather forecast analysis using the [Open-Meteo Forecast API](https://open-meteo.com/en/docs) (16-day forecasts instead of historical data)

## Apply to Other Domains  
- **Genomics**: Connect to sequence databases like the [Sequence Read Archive](https://www.ncbi.nlm.nih.gov/sra/docs/)
- **Geospatial**: Integrate satellite imagery from [Landsat Collection 2](https://www.usgs.gov/landsat-missions/landsat-commercial-cloud-data-access)
- **Business**: Link to sales data, market research, or customer analytics APIs
- **Finance**: Access real-time market data, economic indicators, or trading APIs
- **Healthcare**: Query medical databases, research papers, or treatment outcome datasets

## Technical Enhancements
- Build standalone MCP servers that other clients can use
- Add authentication and rate limiting for production use
- Implement data caching and optimization strategies
- Create interactive web interfaces for non-technical users

The MCP patterns you've learned here will work with any external data source. The possibilities are truly limitless! ✨

**Happy data exploring!**